In [2]:
import dice_ml
from dice_ml.utils import helpers # helper functions
from sklearn.model_selection import train_test_split

dataset = helpers.load_adult_income_dataset()
target = dataset["income"] # outcome variable
train_dataset, test_dataset, _, _ = train_test_split(dataset,
                                                     target,
                                                     test_size=0.2,
                                                     random_state=0,
                                                     stratify=target)
# Dataset for training an ML model
d = dice_ml.Data(dataframe=train_dataset,
                 continuous_features=['age', 'hours_per_week'],
                 outcome_name='income')

# Pre-trained ML model
m = dice_ml.Model(model_path=dice_ml.utils.helpers.get_adult_income_modelpath(),
                  backend='TF2', func="ohe-min-max")

In [5]:
# DiCE explanation instance
exp = dice_ml.Dice(d,m)

# Generate counterfactual examples
query_instance = test_dataset.drop(columns="income")[0:1]

In [6]:
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite")

100%|██████████| 1/1 [00:01<00:00,  1.67s/it]


In [8]:
# Visualize counterfactual explanation
dice_exp.visualize_as_dataframe()

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,29,Private,HS-grad,Married,Blue-Collar,White,Female,38,0



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,38.0,Private,HS-grad,Married,Blue-Collar,White,Female,80.0,1
1,90.0,Private,HS-grad,Married,Sales,White,Female,38,1
2,29,Private,HS-grad,Married,White-Collar,White,Female,77.0,1
3,29,Private,HS-grad,Married,Blue-Collar,White,Female,72.0,1
